ELEC-E5550 - Statistical Natural Language Processing
# SET 3: N-gram language models

# Released: 22.1.2019
# Deadline: 5.2.2019

After completing this assignment, you'll understand how statical language models can be estimated. You'll be able to evaluate them and to generate text using them.

KEYWORDS:

- Language models
- N-grams
- Perplexity
- Additive smoothing

### Data
Jane Austen's Pride and Prejudice, in `/coursedata/pride-and-prejudice.txt`

### Libraries
In this task you'll use the following libraries:
- [random](https://docs.python.org/3/library/random.html) -  is a module that implements pseudo-random number generators for various distributions.
- [maths](https://docs.python.org/3/library/math.html) - is a module providing access to the mathematical functions defined by the C standard.
- [NLTK](http://www.nltk.org) — a platform to work with human language data.

## TASK 1
## Warm up: estimate probability scores by hand
As you've already noticed in the first assignment, different language sequences are not equally likely to occur. We've only looked at word frequencies and at frequencies of letter sequences, but what if we want to estimate how probable it is to see some sentence? Well, for this purpose you'll need a **language model**.

A **language model** predicts the following word (or other symbol) given the observed history.
$P(w_i| w_{i−1} . . . w_0)$.
**Language models** are useful, for example, in the task of speech recognition. They help to distinguish between homophones (words that sound the same but have different meanings), for example, _"to"_ and _"too"_ in _"I love you too"_ and _"I love you to death"._

### What is an n-gram language model?
An **n-gram language model** approximates the probability of a word given all the previous words by using only the conditional probability of the $N-1$ preceding words. This approach is based on the Markov assumption: the next word depends only on a fixed-size window of previous words and not on the whole history: $P(w_i| w_{i−1} . . . w_{i-h})$
<img src= "../../../coursedata/notebook_illustrations/n_gram.png">
To use an n-gram language model, we need estimates of the probability of seeing a particular word given the recent
history. For instance, in the 3-gram model, the history is 2 words.

* 3-gram: (say hello **to**) 
* 2 words of history
* 1 word for **prediction**


An intuitive way to estimate probabilities is calculating **maximum likelihood estimation** (MLE). To get the MLE estimate of an n-gram model we get counts from a corpus and normalize these counts to lie between 0 and 1. In the case of bigram language model, when we want to get a probability of some particular bigram $P(w_n|w_{n−1})$, we’ll compute the count of the bigram $C(w_{n−1}w_n)$ and normalize it by the sum of all the bigrams that start with the same first word $w_{n−1}$. It is easy to notice that the sum of all bigram counts that have the same fist word is simply the unigram count for that word $w_{n−1}$. Thus, we get:

$P(w_n|w_{n−1}) = \frac{C(w_{n−1}w_n)}{C(w_{n−1})}$, where
$C$ tells the number of occurrences in the training
set.

The probability of the entire word sequence can be computed using **the chain rule of probability**. And considering the bigram assumption, it is:

$P(w_1^n) ≈ \prod_{k=1}^{n}P(w_k|w_1^{k−1}) ≈ \prod_{k=1}^{n}P(w_k|w_{k−1}) $

<img src= "../../../coursedata/notebook_illustrations/n_gram_chain.png">

### Compute MLE estimates for a bigram model by hand
## 1.1

Everything is easy peasy as a concept, but there are some things to consider in practice. We need to augment our corpus by giving each sentence a special start-symbol **&lt;s>**, and a special end-symbol **&lt;/s>**. The start-symbol provides the context for the first word, so we know the words that are more likely to begin a sentence. The end-symbol makes it possible for the n-gram grammar to be a true probability distribution. Without an end-symbol, the sentence probabilities for all sentences of a given length would sum to one. 

Given the corpus below with all its start and end symbols,

1. "&lt;s> say hello to my little friend &lt;/s>"
2. "&lt;s> say hello &lt;/s>"
3. "&lt;s> say it to my hand &lt;/s>"

etimate probabilities of the following bigrams:
* P(hello|say)
* P(my|to)
* P(to|hello)
* P(say|&lt;s>)
* P(my|say)

Write your estimates as values into the variables in the cell below.

In [1]:

p_say_hello = 2/3 # type in the answer as a number between 0 and 1. For example:
# p_say_hello = 1.
p_to_my = 2/3 # type in the answer as a number between 0 and 1. For example:
# p_to_my = 1.
p_hello_to = 1/3 # type in the answer a number between 0 and 1. For example:
# p_hello_to = 1.
p_start_say = 1 # type in the answer a number between 0 and 1. For example:
# p_start_say = 1.
p_say_my = 0 # type in the answer a number between 0 and 1. For example:
# p_say_my = 1.

#Remember to remove the raise NotImplementedError line:
# YOUR CODE HERE


In [2]:
### This cell contains hidden tests for the correct answers.
from numpy.testing import assert_almost_equal


## TASK 2
## Building an n-gram model
### 2.1 pad sentences with start and end symbols

As we already mentioned, when dealing with language, it is very important to know what words tend to start and end sentences. To learn this with n-grams, we create special symbols of sentence beginning _"&lt;s>"_ and sentence end _"&lt;/s>"_. As a pre-processing step, you need to "pad" your sentences with these symbols and then create n-grams.

Write a function that takes in a list of tokens, puts the needed amount of special start and end symbols in it so that we would definitely know what token or token sequence starts and ends this list.

In [3]:
def pad(token_list, n):
    """
    this function takes in a list of tokens and pads them with special symbols
    
    INPUT:
    token_list - a list of tokens to be padded
    n - the length of a token sequence
    OUTPUT:
    padded_list - a padded list of tokens
    """
    start = "<s>"
    end = "</s>"
    # YOUR CODE HERE
    padded_list = [start]*(n-1) + token_list + [end]*(n-1)
    
    return padded_list

In [4]:
from nose.tools import assert_equal

assert_equal(pad(['a','b','c'],2), ['<s>', 'a', 'b', 'c', '</s>'])
assert_equal(pad(['a','b','c'],3), ['<s>', '<s>', 'a', 'b', 'c', '</s>', '</s>'])


### 2.2 convert a sentence into n-grams

Let's go on and create a function that takes a list of tokens and creates an array of n-grams. The sequence of tokens is very important, so make sure your n-grams are immutable (tuples).

In [5]:
def make_n_grams(token_list, n):
    """
    this function takes in a list of tokens and forms a list of n-grams (tuples)
    
    INPUT:
    token_list - a list of tokens to be converted into n-grams
    n - the length of a token sequence in an n-gram
    OUTPUT:
    n_grams - a list of n-gram tuples
    """
    if n > len(token_list):
        print("The N is too large.")
    # YOUR CODE HERE
    n_grams = [tuple(token_list[i:i+n]) for i in range(len(token_list)-n+1)]
    
    return n_grams

In [6]:
from nose.tools import assert_equal

assert_equal(make_n_grams(['a','b','c','d','e'], 2), [('a', 'b'), ('b', 'c'), ('c', 'd'), ('d', 'e')])
assert_equal(make_n_grams(['a','b','c','d','e'], 3), [('a', 'b', 'c'), ('b', 'c', 'd'), ('c', 'd', 'e')])


### 2.3 get n-gram counts
Now we can collect statistics from our training corpus.

In [7]:
def get_counts(sentence_list, n):
    """
    this function takes in a list of tokenized and padded sentences,
    forms a list of n-grams and gives out a dictionary 
    with counts for every seen n-gram
    
    INPUT:
    sentence_list - a list of tokenized and padded sentences to be converted into n-grams
    n - the length of a token sequence
    OUTPUT:
    n_gram_dict - a dictionary of n_gram history parts as keys, 
    where their values are a dictionary of all continuations and their counts
    {('a',): {'b': 3 'c': 4}
    
    """
    #YOUR CODE HERE
    n_gram_dict = {}
    ngrams_list = [make_n_grams(sentence, n) for sentence in sentence_list]
    for ngrams in ngrams_list:
        for ngram in ngrams:
            if ngram[:-1] not in n_gram_dict.keys():
                n_gram_dict[ngram[:-1]] = {ngram[-1]: 1}
            else:
                if ngram[-1] not in n_gram_dict[ngram[:-1]].keys():
                    n_gram_dict[ngram[:-1]][ngram[-1]] = 1
                else:
                    n_gram_dict[ngram[:-1]][ngram[-1]] += 1
    return n_gram_dict

In [8]:
dummy_corpus = [["say", "hello", "to", "my", "little", "friend"],
                ["say", "hello"],
                ["say", "it", "to", "my", "hand"]]


dummy_corpus_padded = [pad(sentence,3) for sentence in dummy_corpus]
dummy_model_3 = get_counts(dummy_corpus_padded, 3)

assert_equal(dummy_model_3[('<s>', '<s>')], {'say': 3})


### 2.4 compute an MLE language model

We already have all the tools to form the items to count in a corpus, now we need to just count them.
We've already looked at the **Maximum Likelihood Estimate** for bigrams. In general case it works like this:

$P(w_n|w^{n−1}_{n−N+1}) = \frac{C(w^{n−1}_{n−N+1}w_n)}{C(w^{n−1}_{n−N+1})}$, where $C$ tells the number of occurrences in the training corpus. Basically, you just divide the observed frequency of a particular n-gram by the observed frequency of its "history" part. 

<img src= "../../../coursedata/notebook_illustrations/mle.png">

* 3-gram: (say hello **to**) 
* 2 words of history
* 1 word for **prediction**

Here, the whole 3-gram is (say hello to), and its history part is (say hello). If some n-gram is absent from the training corpus, its MLE estimate would be zero.

In [9]:
def score_mle(model_counts, n_gram, **scoring_parameters):
    """
    this function takes in a dictionary of ngram counts and some n-gram,
    and gives out an MLE estimate for this n-gram

    INPUT:
    model_counts - a dictionary of n_gram history parts as keys,
    where their values are a dictionary of all continuations and their counts
        {('a',): {'b': 3 'c': 4}
    n_gram - an ngram as tuple
    scoring_parameters - additional, optional scoring parameters, making the function interface generic,
        however not used here.

    OUTPUT:
    mle_score - MLE score for the n_gram
    """

    n = len(list(model_counts.keys())[0]) + 1
    # YOUR CODE HERE
    if n_gram[:-1] not in model_counts.keys() or n_gram[-1] not in model_counts[n_gram[:-1]].keys():
        return 0
    else:
        n_gram_score = model_counts[tuple(n_gram[:-1])][n_gram[-1]] / sum(model_counts[tuple(n_gram[:-1])].values())
    return n_gram_score

In [10]:
dummy_corpus = [["say", "hello", "to", "my", "little", "friend"],
                ["say", "hello"],
                ["say", "it", "to", "my", "hand"]]


dummy_corpus_padded = [pad(sentence,3) for sentence in dummy_corpus]
dummy_model_3 = get_counts(dummy_corpus_padded, 3)

assert_almost_equal(score_mle(dummy_model_3, ("<s>","say", "it")),0.33, 2)
assert_almost_equal(score_mle(dummy_model_3, ("say","it", "friend")), 0, 2)
assert_almost_equal(score_mle(dummy_model_3, ("say","wow", "now")), 0, 2)


### 2.5  Smoothing counts
Simple N-gram models have one very serious limitation: they are unable to give a probability estimate not only for n-grams with new out-of-vocabulary words but also for the n-grams with known vocabulary but unseen during training. The higher the $N$, the sparser the data, and the more zero counts there will be. To overcome this problem, we need to redistribute some probability mass from more frequent events and give it to the events we’ve never seen. These techniques are called **smoothing** or **discounting**. 

In this assignment, you will implement **Additive Smoothing**. In practice, it's not the most successful smoothing method, but it will give you an understanding of the intuition behind even more elaborate techniques.

The idea of this method is that we pretend we’ve seen each n-gram $\delta$ times more than we actually have. Typically, $0 < \delta ≤ 1$. This way, we just add $\delta$ to the original counts.

$P(w_i|w^{i-1}_{i-n+1}) = \frac{\delta + C(w_{i-n+1}^i)}{\delta|V|+C(w^{i-1}_{i−n+1})}$

The $|V|$ is the (prediction) vocabulary of all possible continuations for an n-gram. It can be any word from a sentence, a special end symbol, but not a special start symbol (because it appears only as an (n-1)-gram).

In [11]:
def score_smoothed(model_counts, n_gram, **scoring_parameters):
    """
    this function takes in a dictionary of ngram counts, some ngram and delta to be added to this ngram's 
    score, and gives out a smoothed estimate for this ngram.
    if some word in an n-gram is unseen during training, the estimate is zero.

    INPUT:
    model_counts - a dictionary of n_gram history parts as keys,
        where their values are a dictionary of all continuations and their counts
        {('a',): {'b': 3 'c': 4}
    n_gram - an ngram as a tuple
    scoring_parameters - additional, optional scoring parameters, which make the function interface generic
        here we will look for scoring_parameters["delta"] - the delta value to be added to the counts
                        and for scoring_parameters["vocab"] -  the vocabulary that can be used as
                        a continuation of an (n-1)-gram.

    OUTPUT:
    smoothed_score - a smoothed score for the n_gram
    """
    delta = scoring_parameters["delta"]
    vocab = scoring_parameters["vocab"]
    context_length = len(next(iter(model_counts)))
    n = context_length + 1  # the ngram length n
    context = n_gram[:context_length]
    word_to_predict = n_gram[-1]
    if any(token not in vocab | {'<s>'} for token in n_gram):
        raise ValueError("Distribution not defined on this n_gram:" + repr(n_gram))

    # YOUR CODE HERE
    if n_gram[:-1] not in model_counts.keys():
        denominator = delta * len(vocab)
        nominator = delta
    else:
        if n_gram[-1] not in model_counts[n_gram[:-1]].keys():
            nominator = delta
        else:
            nominator = delta + model_counts[tuple(n_gram[:-1])][n_gram[-1]]
        denominator = delta * len(vocab) + sum(model_counts[tuple(n_gram[:-1])].values())

    smoothed_score = nominator / denominator
    return smoothed_score

In [12]:
import itertools
dummy_corpus = [["say", "hello", "to", "my", "little", "friend"],
                ["say", "hello"],
                ["say", "it", "to", "my", "hand"]]


dummy_corpus_padded = [pad(sentence,3) for sentence in dummy_corpus]
dummy_model_3 = get_counts(dummy_corpus_padded, 3)
dummy_model_vocab = set(word for word in itertools.chain(*dummy_corpus)) | {'</s>'}
            
assert_almost_equal(score_smoothed(dummy_model_3, 
                                   ("<s>","say", "it"), 
                                   delta=1, 
                                   vocab=dummy_model_vocab), 0.16, 2)

assert_almost_equal(score_smoothed(dummy_model_3, 
                                   ("say","it", "friend"), 
                                   delta=1, 
                                   vocab=dummy_model_vocab), 0.1, 2)


## TASK 3
## How to evaluate a language model?

The best way to evaluate a language model is to look at its performance in the intended application. Unfortunately, it is usually time-consuming to run the whole system just to test the language model parameters. Instead, we can measure how well an n-gram model predicts unseen data called the test set or test corpus. The higher the probability that the model assigns to the test set, the better this model performs. 

### 3.1 probability of a sentence
Let's compute the probability of a test sentence. Remember we were talking about **the chain rule of probability**? This is exactly the right moment to use it.

The language model probabilities are usually represented and computed in log format (as log probabilities). It has one very simple explanation: probabilities are small numbers between 0 and 1, and when we multiply these probabilities, we get even smaller product. We take a log of probabilities just to avoid numerical underflow.

Adding in log space is equivalent to multiplying in linear space, so we combine log probabilities by adding them.

In [13]:
import math
def sentence_logprob(sentence, model_counts, score_function, **scoring_parameters):
    """
    this function takes in a tokenized sentence, language model counts, and a score function.
    it pads the sentence with special symbols,
    and gives out its probability according to the n-gram model and the scoring method

    INPUT:
    sentence - a tokenized sentence
    model_counts - a dictionary of n_gram history parts as keys,
    where their values are a dictionary of all continuations and their counts
    ('a',): {'b': 3 'c': 4}
    score_function - a function which takes a dictionary of counts, an n-gram, and possible additional 
    parameters, and produces a score for the last token of the n-gram, given the rest as context
    scoring_parameters - additional, optional scoring parameters, passed to score_function
        like this: score_function(model_counts, ngram, **scoring_parameters)
    OUTPUT:
    logprob - a log probability score of a sentence
    """
    context_length = len(next(iter(model_counts)))
    n = context_length + 1  # the ngram length n
    sentence_grams = make_n_grams(pad(sentence, n), n)
    logprob = 0

    # YOUR CODE HERE
    for ngram in sentence_grams:
        mle = score_function(model_counts, ngram, **scoring_parameters)
        if not mle:
            return -float('inf')
        logprob += math.log(mle)
    return logprob


In [14]:
dummy_corpus = [["say", "hello", "to", "my", "little", "friend"],
                ["say", "hello"],
                ["say", "it", "to", "my", "hand"]]


dummy_corpus_padded = [pad(sentence,2) for sentence in dummy_corpus]
dummy_model_2 = get_counts(dummy_corpus_padded, 2)
dummy_model_vocab = set(word for word in itertools.chain(*dummy_corpus)) | {'</s>'}

dummy_test_sentence1 = ["say", "hello", "to", "my", "hand"]
dummy_test_sentence2 = ["say", "hello", "to", "my", "little", "hand"]

assert_almost_equal(sentence_logprob(dummy_test_sentence1, dummy_model_2, score_mle), -1.791, 2)
assert_almost_equal(sentence_logprob(dummy_test_sentence2, dummy_model_2, score_mle), -float('inf'), 2)

assert_almost_equal(sentence_logprob(dummy_test_sentence1, dummy_model_2, score_smoothed, delta=1, vocab=dummy_model_vocab), -8.803,2)
assert_almost_equal(sentence_logprob(dummy_test_sentence2, dummy_model_2, score_smoothed, delta=1, vocab=dummy_model_vocab), -11.106,2)



In practice, the raw probability is not used as a metric for evaluating language models. 
We can look at **perplexity** and **out-of-vocabulary rate** (OOV rate) instead. 
### 3.2 perplexity by hand

**Perplexity** score (PP) tells us how uncertain the model is when predicting the words in the test data $W$. You can think of it as the weighted average branching factor of a language. The branching factor of a language is the number of possible next words that can follow any word. Technically, the perplexity of a language model on some test set is the inverse probability of the test set, normalized by the number of tokens in it.

$PP(W) = \sqrt[N]{\prod_{i=1}^{N}\frac{1}{P(w_i|w_1...w_{i-1})}}$
 
In our case, let's think of $N$ as of a number of n-grams the test set contains.

Let's look at the dummy language where you only have 4 letters (a, b, c, d). These letters in all possible texts appear with equal probability $P = \frac{1}{4}$. You have a 5 letter sentence composed in this language. What is the perplexity of this dummy language as a unigram model? 

Write your answer as a value into the variable in the cell below.

In [15]:
dummy_ppl = 4 # type in the answer, and remove the raise NotImplementedError line.
# YOUR CODE HERE

In [16]:
### This cell contains hidden tests for the correct answers.


### 3.4 perplexity as a function
Write a function for calcualting the perplexity of a test corpus. When the model encounters an unseen n-gram, its perplexity should be infinity. 

In [17]:
def perplexity(text, model_counts, score_function, **scoring_parameters):
    """
    this function takes in test text and the n-gram model counts and gives out the perplexity of this 
    n-gram model

    INPUT:
    text - a list of lists of tokenized sentences
    model_counts - a dictionary of n_gram history parts as keys,
    where their values are a dictionary of all continuations and their counts
    {('a',): {'b': 3 'c': 4}
    score_function - a function which takes a dictionary of counts, an n-gram, and possible additional 
    parameters, and produces a score for the last token of the n-gram, given the rest as context
    scoring_parameters - additional, optional scoring parameters, passed to score_function
        like this: score_function(model_counts, ngram, **scoring_parameters)

    OUTPUT:
    ppl - a perplexity score of a sentence
    """

    context_length = len(next(iter(model_counts)))
    n = context_length + 1  # the ngram length n
    logprob = 0
    num_predictions = 0  # NOTE: The number of predictions per sentence is len(sentence) + context_length
    # YOUR CODE HERE
    corpus_padded = [pad(sentence, n) for sentence in text]
    n_grams = [make_n_grams(sentence, n) for sentence in corpus_padded][0]
    prob = 1
    num_predictions = len(n_grams)
    for n_gram in n_grams:
        if not score_function(model_counts, n_gram, **scoring_parameters):
            return float('inf')
        else:
            prob *= 1 / score_function(model_counts, n_gram, **scoring_parameters)
    ppl = prob ** (1 / num_predictions)
    return ppl

In [18]:
dummy_corpus = [["say", "hello", "to", "my", "little", "friend"],
                ["say", "hello"],
                ["say", "it", "to", "my", "hand"]]


dummy_corpus_padded = [pad(sentence,3) for sentence in dummy_corpus]
dummy_model_3 = get_counts(dummy_corpus_padded, 3)
dummy_model_vocab = set(word for word in itertools.chain(*dummy_corpus)) | {'</s>'}

dummy_test_corpus1 = [["say", "hello", "to", "my", "hand"]]
dummy_test_corpus2 = [["say", "hello", "to", "my", "little", "hand"]]

assert_almost_equal(perplexity(dummy_test_corpus1, dummy_model_3, score_mle), 1.2917, 2)
assert_equal(perplexity(dummy_test_corpus2, dummy_model_3, score_mle), float('inf'))

assert_almost_equal(perplexity(dummy_test_corpus1, 
                               dummy_model_3, 
                               score_smoothed, 
                               delta=1,
                               vocab=dummy_model_vocab), 4.6266, 2)
assert_almost_equal(perplexity(dummy_test_corpus2, 
                        dummy_model_3, 
                        score_smoothed, 
                        delta=1,
                        vocab=dummy_model_vocab), 5.4829, 2)

## TASK 5 
## Generate text
### 5.1 visualize the model
Another useful way to make a sanity check of how model is performing, is by generating sentences with it.

1. Start with the appropriate number (n-1) of start symbols _"&lt;s>"_, e.g. ("&lt;s>","&lt;s>") for trigrams.
2. Generate tokens one at a time. Given the context, you get a probability distribution over the next word. Sample from that distribution. NOTE: For sampling, use random.choices(), calling it exactly once per token.
3. Stop generating when the end symbol is produced.

HINT: Use random.choices()

NOTE: Although it should be possible to make the pseudorandom behaviour reproducible by setting a seed,
    we were not satisfied with the robustness of that solution. Therefore, the generation functions are not
    autograded. However, they are used later when working with real data.

### 5.1.1 MLE text generation
First we will implement text generation for the MLE case. In this case, you can simply use the true counts of each n-gram as the weights for random.choices()

In [19]:
import random

def generate_text_mle(model_counts):
    """
    this function takes in the n-gram model and produces text until the end symbol is generated.

    INPUT:
    model_counts - a dictionary of n_gram history parts as keys,
    where their values are a dictionary of all continuations and their counts
    ('a',): {'b': 3 'c': 4}
    OUTPUT:
    sentence - a sentence generated by model as a list of tokens.
    """
    context_length = len(next(iter(model_counts)))
    n = context_length + 1  # the ngram length n
    start = tuple(['<s>'] * (n - 1))
    end = '</s>'
    sentence = list(start)

    while sentence[-1] != end:
        # YOUR CODE HERE
        n_grams = []
        n_word = list(model_counts[tuple(sentence[-2:])].keys())
        for key in n_word:
            n_grams.append((sentence[-2], sentence[-1], key))
        p = [score_mle(model_counts, n_gram) for n_gram in n_grams]
        sentence.append(*random.choices(n_word, weights=p))

    # Strip the padding:
    sentence = sentence[n - 1:-1]
    return sentence

In [20]:
dummy_corpus = [["say", "hello", "to", "my", "little", "friend"],
                ["say", "hello"],
                ["say", "it", "to", "my", "hand"]]

dummy_corpus_padded = [pad(sentence,3) for sentence in dummy_corpus]
dummy_model_3 = get_counts(dummy_corpus_padded, 3)

print(generate_text_mle(dummy_model_3))

['say', 'hello']


### 5.1.2 Additive smoothing generation

Now we'll implement generating text from a smoothed model. In this case, you can use the modified counts of each n-gram as the weights. Note that in this case, you need to know the vocabulary of the model, so it is given to the function as an argument.

NOTE: vocab should include the end of sentence tag!

In [21]:
def generate_text_smoothed(model_counts, delta, vocab):
    """
    this function takes in the n-gram model and produces text until the end symbol is generated.

    INPUT:
    model_counts - a dictionary of n_gram history parts as keys,
    where their values are a dictionary of all continuations and their counts
    ('a',): {'b': 3 'c': 4}
    delta - the delta value to be added to the counts
    vocab - a set of words in the vocabulary
    OUTPUT:
    sentence - a sentence generated by model as a list of tokens.
    """
    context_length = len(next(iter(model_counts)))
    n = context_length + 1  # the ngram length n
    start = tuple(['<s>'] * (context_length))
    end = '</s>'
    sentence = list(start)

    while sentence[-1] != end:
        curr_context_counts = {}
        try:
            curr_context_counts.update(model_counts[tuple(sentence[-context_length:])])
        except KeyError:  # This context was not seen once
            pass
        # NEXT: Add smoothing delta, then generate the next token.
        # YOUR CODE HERE
        vocab_list = list(vocab)
        if end not in vocab_list:
            vocab_list.append(end)
        n_grams = []
        for word in vocab:
            n_grams.append((sentence[-2], sentence[-1], word))
        p = [score_smoothed(model_counts, n_gram, delta=delta, vocab=vocab) for n_gram in n_grams]
        sentence.append(*random.choices(vocab_list, weights=p))

    # Strip the padding:
    sentence = sentence[context_length:-1]
    return sentence

In [22]:
dummy_corpus = [["say", "hello", "to", "my", "little", "friend"],
                ["say", "hello"],
                ["say", "it", "to", "my", "hand"]]


dummy_corpus_padded = [pad(sentence,3) for sentence in dummy_corpus]
dummy_model_3 = get_counts(dummy_corpus_padded, 3)
dummy_model_vocab = set(word for word in itertools.chain(*dummy_corpus)) | {'</s>'}


print(generate_text_smoothed(dummy_model_3, 
                       delta=0.1,
                       vocab=dummy_model_vocab))

print(generate_text_smoothed(dummy_model_3,
                       delta=0.4,
                       vocab=dummy_model_vocab))


['say', 'say', 'it', 'to', 'my', 'hand']
['it', 'it', 'say', 'say']


## TASK 6
## Working with real text data

Let's try to model some real data. In this assignment we are working with Jane Austen's novel Pride and Prejudice, which is provided Project Gutenberg [here](http://www.gutenberg.org/ebooks/1342). In the version in the coursedata directory, the text has had some ebook disclaimers removed, as they were certainly not written by Jane Austen.

First, we'll tokenize the corpus. As you remember from the Intro assignment, tokenization, processing raw text into a usable format, is not trivial. Here, we'll again use tokenizers from nltk. 

We'll need to split the data into sentences. The nltk's PunktSentenceTokenizer uses "an unsupervised algorithm" to find sentence boundaries, and a pretrained version is available for English. You may be prompted to download it. Though typically sentence boundaries are marked with hard punctuation, particularly the full stop is also found elsewhere, so the task is not at all trivial.

To tokenize the sentences, we'll use the same TreebankWordTokenizer as in the Intro. This will not for example lowercase the data, which might not be ideal for all applications, but here we have no particular use case in mind, so the TreebankWordTokenizer will do. Note that this means that capitalized and lowercase versions of words are now modeled as two separate things.

In [23]:
import nltk.tokenize
import nltk.util

with open("/coursedata/pride-and-prejudice.txt") as fi:
    janeausten_raw = fi.read()

sentence_tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer() # Splits a long text into sentences
word_tokenizer = nltk.tokenize.TreebankWordTokenizer()
word_detokenizer = nltk.tokenize.treebank.TreebankWordDetokenizer() # Reverses tokenization
janeausten_sentences = list(sentence_tokenizer.sentences_from_text(janeausten_raw))
janeausten_tokenized = list(word_tokenizer.tokenize(sentence) for sentence in janeausten_sentences)

print("Let's see an example sentence:")
print(janeausten_tokenized[1000])

Let's see an example sentence:
['Bennet', ',', 'with', 'little', 'cessation', ',', 'of', 'his', 'house', 'and', 'garden', 'at', 'Hunsford', '.']


### Train-test split

We'll divide the data into training and test sets. The training set is used to create the model, and the test set used for evaluation. Here, we'll randomly select 10% of the sentences to be used as the test set. This way, the training data matches the test data very well.

In [24]:
test_split_index = round(0.9 * len(janeausten_tokenized))
random.seed(808)
janeausten_tokenized_randperm = random.sample(janeausten_tokenized, len(janeausten_tokenized))
janeausten_train = janeausten_tokenized_randperm[:test_split_index]
janeausten_test = janeausten_tokenized_randperm[test_split_index:]

print("Training set has", len(janeausten_train), "sentences, and the test set", len(janeausten_test), "sentences")

Training set has 5194 sentences, and the test set 577 sentences


### Vocabulary size

If we just use the full vocabulary of the training data, we will include some rare words, 
while excluding other words. One way to deal with this is to limit the vocabulary to some most common words. 
Everything else will be an unknown token, dealt with, together, as the "&lt;UNK&gt;" token.


We will limit the vocabulary to the 1000 most common words in the training set. Additionally we include the end of sentence tag and the unknown word tag, because we want to predict them.

In [25]:
from collections import Counter
import itertools
janeausten_unigram_counts = Counter(itertools.chain.from_iterable(janeausten_train)).most_common()

print("The 10 most common tokens:\n", "\n".join(word for word, freq in janeausten_unigram_counts[:10]))
print()
print("The 990-1000 most common tokens:\n", "\n".join(word for word, freq in janeausten_unigram_counts[990:1000]))

janeausten_vocab_1k = set(word for word, freq in janeausten_unigram_counts[:1000]) | {"</s>", "<unk>"}


The 10 most common tokens:
 ,
.
to
the
of
and
her
I
a
was

The 990-1000 most common tokens:
 By
application
fancy
congratulations
comparison
try
rich
contempt
distance
sooner


### 6.1  replace tokens not in the vocabulary

Implement the replaing of tokens which are not in the vocabulary (out-of-vocabulary words, OOVs). They are to be replaced with the unknown token "&lt;unk&gt;". Fill in the following function.

In [26]:
def replace_oovs(vocab, data, unk="<unk>"):
    """
    vocab: set of tokens
    data: list of lists, i.e. list of sentences, which are lists of tokens
    unk: token to replace tokens which are not in the vocabulary
    
    This function replaces all tokens not in the vocabulary with the unknown token
    """
    # YOUR CODE HERE
    data_oovs_replaced = list(data)
    for i, sentence in enumerate(data_oovs_replaced):
        for j, token in enumerate(sentence):
            if token not in vocab:
                data_oovs_replaced[i][j] = unk

    return data_oovs_replaced

In [27]:
from nose.tools import assert_equal
assert_equal(replace_oovs({"a","b","c"}, [["a", "b"],["a","b","c","d"]]), [['a', 'b'], ['a', 'b', 'c', '<unk>']])


Now we'll apply the filter to the Jane Austen data.

In [28]:
janeausten_train_1k = replace_oovs(janeausten_vocab_1k, janeausten_train)
janeausten_test_1k = replace_oovs(janeausten_vocab_1k, janeausten_test)

print("Let's see an example:")
print(word_detokenizer.detokenize(janeausten_train_1k[106]))

Let's see an example:
She was received, however, very <unk> by them; and in their brother ’ s manners there was something better than politeness; there was good humour and kindness.


### MLE model

First we'll train a maximum likelihood estimate trigram model. Now that we're using a larger dataset, the estimation might take some time (a minute).

In [29]:
# let's pad the training corpus 
janeausten_train_1k_padded = [pad(sent, n=3) for sent in janeausten_train_1k]

In [30]:
# now let's get the n-gram counts
janeausten_train_1k_counts = get_counts(janeausten_train_1k_padded, n=3)

However, even though any unseen token is accounted for by the unknown token, we still get infinite perplexity:

In [31]:
perplexity(janeausten_test_1k, janeausten_train_1k_counts, score_mle)

inf

### Smoothed model

Now, we'll smooth our counts with a few different delta values and look at perplexities. This will also take some time.

Finally, we can get some type of proper perplexity value.

In [32]:
print(perplexity(janeausten_test_1k, janeausten_train_1k_counts, score_smoothed, delta=1, vocab=janeausten_vocab_1k))
print(perplexity(janeausten_test_1k, janeausten_train_1k_counts, score_smoothed, delta=0.01, vocab=janeausten_vocab_1k))

284.09760341164673
62.943456533810746


### TASK: Generate sentences, comment on differences

Let's see what kinds of sentences do the two types of models generate.
Generate some sentences, and comment on the differences.

- Why do the smoothed models produce such long walls of text?
    - If we backed off to lower order models, would that help?
- Which model generates text, that looks most like the original? Why?
- Which model do you think would be the best language model for e.g. optical character recognition of Jane Austen's hand written notes, and why?

In [33]:
print("Sentences from the MLE model:")
for i in range(5):
    print(word_detokenizer.detokenize(generate_text_mle(janeausten_train_1k_counts)))


Sentences from the MLE model:
“ Oh!
The reason why all this <unk>.
But <unk> ’ <unk>.
“ You <unk>, and <unk> of Lydia on this subject, as to me about him for us; was <unk> settled, that he was again <unk> in the <unk> feelings were yet more, till the following hour.
I, ” replied Elizabeth with some <unk>; but there were <unk> to his real character, and <unk> the <unk> of <unk> into the <unk>; that had not herself <unk> to be expected.


In [34]:
print("A sentence from the low delta smoothed model:")
print(word_detokenizer.detokenize(generate_text_smoothed(janeausten_train_1k_counts, delta=0.01, vocab=janeausten_vocab_1k)))


A sentence from the low delta smoothed model:
The two suffer what journey Mrs acquainted play madam once happiness ask live disposed For dancing wishes housekeeper convinced knew astonishment concerned terms comprehend so convinced me appear cousin instantly praise Their Chapter sometimes praise After how) So stay contempt nothing leave removed doing above made perhaps Forster easily avoid indifference Chapter heart conversation They success greatly should express dearest subject consent miles unable thus ago already eyes has character kindness might recollected estate elegant journey form met girls silence at these doing being prevent marry neighbourhood pounds convinced As From am affection son learnt sensible sat return reached called Miss find above desire help case either expression Are settled child excuse more letter reached bring little lost gave credit that glad not too high! beauty taken scheme _my_ address against change cold Forster voice attention _her_ between supposing p

In [35]:
print("A sentence from the high delta smoothed model:")
print(word_detokenizer.detokenize(generate_text_smoothed(janeausten_train_1k_counts, delta=1.0, vocab=janeausten_vocab_1k)))

A sentence from the high delta smoothed model:
<unk> Are afraid why speaking hand _your_ remained mother credit child disappointment minutes wishing Such it send opening bring reached arrived scarcely immediately How before company Oh natural neither dear greater _her_ de care avoid she length seemed duty wishing soon left expectation then (dislike intelligence chose short standing but call times library observed sort wish fortnight stay repeated Kitty case got advantage certainly such certain account almost speaking Indeed good Caroline father especially particular none speaking made resentment any arrived they dared Rosings comparison pass compliment saw recollected able told readily this part eye sent doing answered affectionate occurred greatest particularly event sisters niece inquiries than quite intended meet ignorant far deal learnt spoke pride report father reason affair repeated from then, politeness far taken sisters future satisfied because satisfied park Well against behin

YOUR ANSWER HERE

i) The smoothed models are capable of producing way linger text as they consider the whole vocabulary instead of only using the existing n-grams, which leaves more options for a follow-up word without hitting the end-token.

ii) The MLE model as it is basically only capable of reproducing n-grmas which makes the text more similar

iii) The smoothed model should perform best as we want to be capable of predicting unseen token/character patterns